# Тест парсеров

In [1]:
from parsers.mpstats import get_mpstats_texts

samples = get_mpstats_texts(limit=20)

ImportError: cannot import name 'get_mpstats_texts' from 'parsers.mpstats' (/Users/akayo/Desktop/ai-seller-rag/parsers/mpstats.py)

In [2]:
samples[0]

'Маркировка товаров на Wildberries\nЗачем нужна и как сделать\nЛогистика на Wildberries: тарифы, доставка и индекс локализации\nРасскажем, как оптимизировать траты\nКомиссии Wildberries для селлеров\nСколько и за что платим в 2025 году\nКак сделать первую поставку на WB\nИщем наиболее выгодные позиции\nКак узнать остатки товара на Wildberries\nИнструкция для селлеров\nПлатная приемка на Wildberries\nЧто это и как отгружать поставки бесплатно\nЧто такое КИЗ товаров на Вайлдберриз\nВ чем особенность маркировки и для чего она нужна\nПриемка товара на Wildberries: как не наделать ошибок\nПоставки, правила упаковки и бесплатная приемка\nОсобенности схемы FBO на Wildberries\nРасскажем о поставках на склад маркетплейса и нюансах модели\nКак сделать расчет поставки на Вайлдберриз\nПланируем поставку на склад маркетплейса\nРаспределительный центр Wildberries\nЧто это такое, где находятся и как работает\nВозврат товара на Wildberries для поставщика\nКак селлеру вернуть продукцию со склада маркет

In [ ]:
from parsers.tg import get_telegram_texts

samples = await get_telegram_texts("https://t.me/itatmisis", limit=20)

In [ ]:
samples[0]

# Обработчик фрагментов тесты

In [20]:
from abc import ABC, abstractmethod
from io import BytesIO

import requests


class PDFExtractor(ABC):
    """Абстрактный класс для извлечения текста из PDF"""

    @abstractmethod
    def extract_text(self, pdf_bytes):
        pass


class PyMuPDFExtractor(PDFExtractor):
    """Экстрактор на основе PyMuPDF (fitz) - самый быстрый и точный"""

    def extract_text(self, pdf_bytes):
        try:
            import fitz
            doc = fitz.open(stream=pdf_bytes, filetype="pdf")
            full_text = ""

            print(f"PyMuPDF: обрабатываем {doc.page_count} страниц")

            for page_num in range(doc.page_count):
                page = doc[page_num]
                text = page.get_text()
                full_text += text + "\n"

                # Прогресс для больших документов
                if (page_num + 1) % 10 == 0:
                    print(f"PyMuPDF: обработано {page_num + 1} страниц")

            doc.close()
            return full_text.strip()

        except Exception as e:
            raise Exception(f"Ошибка PyMuPDF: {e}")


class PdfPlumberExtractor(PDFExtractor):
    """Экстрактор на основе pdfplumber - хорош для таблиц"""

    def extract_text(self, pdf_bytes):
        try:
            import pdfplumber
            full_text = ""

            with pdfplumber.open(BytesIO(pdf_bytes)) as pdf:
                print(f"pdfplumber: обрабатываем {len(pdf.pages)} страниц")

                for i, page in enumerate(pdf.pages):
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                    if (i + 1) % 10 == 0:
                        print(f"pdfplumber: обработано {i + 1} страниц")

            return full_text.strip()

        except Exception as e:
            raise Exception(f"Ошибка pdfplumber: {e}")


class PyPDF2Extractor(PDFExtractor):
    """Экстрактор на основе PyPDF2 - базовый вариант"""

    def extract_text(self, pdf_bytes):
        try:
            import PyPDF2
            pdf_file = BytesIO(pdf_bytes)
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            full_text = ""

            print(f"PyPDF2: обрабатываем {len(pdf_reader.pages)} страниц")

            for i, page in enumerate(pdf_reader.pages):
                text = page.extract_text()
                if text:
                    full_text += text + "\n"

                if (i + 1) % 10 == 0:
                    print(f"PyPDF2: обработано {i + 1} страниц")

            return full_text.strip()

        except Exception as e:
            raise Exception(f"Ошибка PyPDF2: {e}")


class WildberriesOfferParserMultiEngine:
    """Парсер оферты Wildberries с поддержкой нескольких PDF-библиотек"""

    def __init__(self, preferred_engine=None):
        self.headers = {"User-Agent": "Mozilla/5.0"}
        self.extractor = None
        self.engine_name = None

        # Определяем порядок предпочтения библиотек
        engines = [
            ("pymupdf", PyMuPDFExtractor, "fitz"),
            ("pdfplumber", PdfPlumberExtractor, "pdfplumber"),
            ("pypdf2", PyPDF2Extractor, "PyPDF2")
        ]

        # Если указан предпочтительный движок
        if preferred_engine:
            for name, extractor_class, import_name in engines:
                if name == preferred_engine.lower():
                    if self._try_import_engine(import_name, extractor_class, name):
                        break

        # Если движок не выбран или не удалось инициализировать, пробуем по порядку
        if not self.extractor:
            for name, extractor_class, import_name in engines:
                if self._try_import_engine(import_name, extractor_class, name):
                    break

        if not self.extractor:
            raise ImportError("Ни одна библиотека для работы с PDF не доступна. "
                            "Установите: pip install PyMuPDF или pip install pdfplumber или pip install PyPDF2")


    def _try_import_engine(self, import_name, extractor_class, engine_name):
        """Пытается импортировать и инициализировать движок"""
        try:
            __import__(import_name)
            self.extractor = extractor_class()
            self.engine_name = engine_name
            print(f"✅ Используется PDF-движок: {engine_name.upper()}")
            return True
        except ImportError:
            print(f"❌ {engine_name.upper()} недоступен")
            return False


    def get_offer_text(self):
        """Загружает и извлекает текст оферты"""
        pdf_url = "https://static-basket-02.wb.ru/vol20/offers/prd/product/latest.pdf"

        try:
            print("📥 Загружаем PDF оферты...")
            response = requests.get(pdf_url, headers=self.headers, timeout=30)
            response.raise_for_status()

            print(f"📊 Размер PDF: {len(response.content) / 1024:.1f} KB")

            print(f"🔧 Извлекаем текст с помощью {self.engine_name.upper()}...")
            text = self.extractor.extract_text(response.content)

            print(f"✅ Извлечено текста: {len(text)} символов")
            return text

        except Exception as e:
            print(f"❌ Ошибка при загрузке или извлечении: {e}")
            return None



# ИСПОЛЬЗОВАНИЕ
if __name__ == "__main__":

    # Вариант 1: Автоматический выбор лучшей библиотеки
    parser = WildberriesOfferParserMultiEngine()

    # Вариант 2: Принудительный выбор конкретной библиотеки
    # parser = WildberriesOfferParserMultiEngine(preferred_engine='pymupdf')
    # parser = WildberriesOfferParserMultiEngine(preferred_engine='pdfplumber')
    # parser = WildberriesOfferParserMultiEngine(preferred_engine='pypdf2')

    # Парсим оферту
    sections = parser.get_offer_text()


✅ Используется PDF-движок: PYMUPDF
📥 Загружаем PDF оферты...
📊 Размер PDF: 927.5 KB
🔧 Извлекаем текст с помощью PYMUPDF...
PyMuPDF: обрабатываем 84 страниц
PyMuPDF: обработано 10 страниц
PyMuPDF: обработано 20 страниц
PyMuPDF: обработано 30 страниц
PyMuPDF: обработано 40 страниц
PyMuPDF: обработано 50 страниц
PyMuPDF: обработано 60 страниц
PyMuPDF: обработано 70 страниц
PyMuPDF: обработано 80 страниц
✅ Извлечено текста: 291424 символов


In [21]:
sections

'---------------------------------------------------------------------------------------------------------------------------------------  \nЕсли у Вас уже есть действующий договор о реализации товаров на сайте wildberries с ООО \n«Вайлдберриз», ИНН 7721546864, то, принимая настоящую оферту и нажимая кнопку «Подтвердить», \nВы выражаете согласие (одобрение) на то, что ООО «Вайлдберриз», ИНН 7721546864, 05.08.2024 \nпередало все права и обязанности по ранее заключенному Вами договору о реализации товаров на \nсайте wildberries в пользу нового оператора сайта ООО «РВБ», ИНН 9714053621, в полном объеме, \nсуществующем на момент передачи, а также поручаете ООО "Вайлдберриз", ИНН 7721546864, \nпередать информацию, необходимую для исполнения Договора, включая персональные данные, в адрес \nООО "РВБ", ИНН 9714053621, а также гарантируете наличие необходимых для этого правовых \nоснований. \n--------------------------------------------------------------------------------------------------------

In [16]:
import re 

pattern = r'\n([^\n]+)\n\d+\.1\. '

matches = re.findall(pattern, sections)
print(matches)

['1. Определения. ', '2. Порядок заключения договора: ', '3. Предмет Договора ', '4. Продажа Товара. ', '5. Цена Товара, отчетность и расчеты. ', '6. Вознаграждение и иные выплаты Продавца. ', '7. Изменение Договора. ', '8. Заключительные положения. ', 'ПРАВИЛА ИСПОЛЬЗОВАНИЯ ПОРТАЛА  ', ' ', 'ПРАВИЛА ПРИЕМКИ И ВОЗВРАТА ТОВАРА ', '«МАРКЕТПЛЕЙС» (FBS) ', 'ПРАВИЛА ОКАЗАНИЯ УСЛУГ  ', 'порядке и на следующих условиях:  ', '15. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО ПРОДВИЖЕНИЮ ТОВАРОВ ', '16. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО РАЗМЕЩЕНИЮ РЕКЛАМНЫХ МАТЕРИАЛОВ  ', 'дальнейшего исполнения обязательств по Договору, Стороны договорились о следующем: ', '20. УДЕРЖАНИЕ ТОВАРОВ. ', '21. ПРАВИЛА ПРОДАЖИ ТРАНСПОРТНЫХ СРЕДСТВ ', '22. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО РАЗМЕЩЕНИЮ ПРЕДЛОЖЕНИЙ О БРОНИРОВАНИИ ТОВАРА. ']


In [17]:
def remove_elements_by_indices(data, indices_to_remove):
    """
    Удаляет элементы из списка data по индексам из indices_to_remove.
    Удаление начинается с конца, чтобы индексы не смещались.
    
    Args:
        data (list): исходный список
        indices_to_remove (list of int): индексы для удаления (начиная с 1)
        
    Returns:
        list: список после удаления указанных элементов
    """
    # Сортируем индексы по убыванию
    indices_sorted = sorted(set(indices_to_remove), reverse=True)
    
    # Преобразуем из 1-базированной нумерации в 0-базированную, проверяем диапазон
    valid_indices = [i-1 for i in indices_sorted if 0 < i <= len(data)]
    
    # Удаляем элементы по индексам, начиная с конца
    for idx in valid_indices:
        del data[idx]
    
    return data

# Пример
sections = ['1. Определения. ', '2. Порядок заключения договора: ', '3. Предмет Договора ',
            '4. Продажа Товара. ', '5. Цена Товара, отчетность и расчеты. ', '6. Вознаграждение и иные выплаты Продавца. ',
            '7. Изменение Договора. ', '8. Заключительные положения. ', '9. Дополнительные условия',
            '10. Исключаемый раздел', '11. Еще раздел',
            '12. Исключаемый раздел', '13. Следующий раздел',
            '14. Исключаемый раздел', '15. Продолжение', '16. Раздел',
            '17. Исключаемый раздел', '18. Другой раздел']

# Индексы для удаления (по номерам элементов в списке, начиная с 1)
remove_nums = [10, 12, 14, 17]

filtered_sections = remove_elements_by_indices(matches.copy(), remove_nums)
print(filtered_sections)


['1. Определения. ', '2. Порядок заключения договора: ', '3. Предмет Договора ', '4. Продажа Товара. ', '5. Цена Товара, отчетность и расчеты. ', '6. Вознаграждение и иные выплаты Продавца. ', '7. Изменение Договора. ', '8. Заключительные положения. ', 'ПРАВИЛА ИСПОЛЬЗОВАНИЯ ПОРТАЛА  ', 'ПРАВИЛА ПРИЕМКИ И ВОЗВРАТА ТОВАРА ', 'ПРАВИЛА ОКАЗАНИЯ УСЛУГ  ', '15. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО ПРОДВИЖЕНИЮ ТОВАРОВ ', '16. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО РАЗМЕЩЕНИЮ РЕКЛАМНЫХ МАТЕРИАЛОВ  ', '20. УДЕРЖАНИЕ ТОВАРОВ. ', '21. ПРАВИЛА ПРОДАЖИ ТРАНСПОРТНЫХ СРЕДСТВ ', '22. ПРАВИЛА ОКАЗАНИЯ УСЛУГ ПО РАЗМЕЩЕНИЮ ПРЕДЛОЖЕНИЙ О БРОНИРОВАНИИ ТОВАРА. ']


In [19]:
#10

pattern = r'\n([^\n]+?)\n\s*\n\d+\.1\.'

matches = re.findall(pattern, sections)
print(matches)

TypeError: expected string or bytes-like object

In [ ]:
#12

pattern = r'\n12\.\s*((?:\n[^\n]+)+)\n12\.1\.'

matches = re.findall(pattern, sections, flags=re.MULTILINE)
print(matches)

['\nПРАВИЛА \nРАСЧЕТА \nВОЗНАГРАЖДЕНИЯ. \nПОРЯДОК \nПРИЕМКИ \nТОВАРА \nДЛЯ \nСПОСОБА \n«МАРКЕТПЛЕЙС» (FBS) ']


In [24]:
#14

pattern = r'\n([\s\S]*?)\nВайлдберриз'

matches = re.findall(pattern, sections)
print(matches)

[]


In [22]:
#17

pattern = r'\n17\.\s*((?:\n(?!Заявка)[^\n]+)+)'

matches = re.findall(pattern, sections, flags=re.MULTILINE)
print(matches)

['\nФОРМА ЗАЯВКИ НА ОКАЗАНИЕ УСЛУГ «ВБ.МЕДИА» ']
